In [2]:
from typing_extensions import Self
import tensorflow as tf
import pandas as pd
import pickle
import numpy as np

In [6]:
class GeneratorPipeline(tf.keras.utils.Sequence):

    def __init__(self, X1, X2, y, batch_size,shuffle=False):
        self.X1 = X1
        self.X2 = X2
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.prepare_training_data()

# encode image using efficientnetb0
# load encoded image

    def prepare_training_data(self):
    
        features_file = '../output/extracted_features/extract_features_6k.pkl'
        file = open(features_file, 'rb')
        features_dict = pickle.load(file)
        file.close()
        
        captions_file = '../output/extracted_features/cap'
        file = open(captions_file, 'rb')
        x2_captions = pickle.load(file)
        file.close()
        
        cap_img_list = x2_captions[0] # first column with list of repeated image id's
        
        X1 = []
        
        for cap_img in cap_img_list:
            img_feature_matrix = features_dict[cap_img][0]
            X1.append(img_feature_matrix)
            
        X1 = np.array(X1)[:100]
        
        X2 = np.array(x2_captions[1]).astype(np.uint32)[:100]
        
        y = np.array([el[0] if len(el)>0 else vocab_size+1 for el in x2_captions[2][:100]])
        
        data = pd.DataFrame({'X1': X1, 'X2': X2, 'y': y})
                                                                    
    def __getitem__(self,index):

        self.load_image("../raw_data/Flickr8k_text/.......")
        

#         X1 = self.X1[index * self.batch_size:(index + 1) * self.batch_size]
#         X2 = self.X2[index * self.batch_size:(index + 1) * self.batch_size]
#         y = self.y[index * self.batch_size:(index + 1) * self.batch_size]

        return [X1,X2], y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
        
    
    def __len__(self):
        return len(self.X1)//self.batch_size